In [1]:
#Install required libraries

!pip install -q transformers datasets evaluate rouge-score sentencepiece kagglehub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.6/355.6 kB 32.8 MB/s eta 0:00:00


In [2]:
#Imports and device setup

import os
import pandas as pd
import torch
from datasets import Dataset
import evaluate


print("Torch available:", torch.cuda.is_available())
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


Torch available: False
Using device: cpu


In [3]:
# Load dataset from KaggleHub
import kagglehub
import os
import pandas as pd
from datasets import Dataset

# Download dataset
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")
print("Path to dataset files:", path)

# List directory contents
print("\nContents of the downloaded directory:")
for root, dirs, files in os.walk(path):
    print("📂", root)
    for f in files:
        print("   └──", f)

# Try to auto-locate the CSVs
def find_csv(base_path, filename):
    for root, _, files in os.walk(base_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

train_file = find_csv(path, "train.csv")
val_file   = find_csv(path, "validation.csv")
test_file  = find_csv(path, "test.csv")

if not train_file or not val_file or not test_file:
    raise FileNotFoundError("❌ Could not locate train/validation/test CSV files. Please check the printed folder structure above.")

# Load into DataFrames
train_df = pd.read_csv(train_file)
val_df = pd.read_csv(val_file)
test_df = pd.read_csv(test_file)

print("\n✅ Dataset loaded successfully!")
print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)

# Convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

# Print a sample
print("\nExample article (truncated):")
print(train_df["article"].iloc[0][:400])
print("\nReference summary:")
print(train_df["highlights"].iloc[0])


100%|██████████| 503M/503M [00:02<00:00, 188MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail/versions/2

Contents of the downloaded directory:
📂 /root/.cache/kagglehub/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail/versions/2
📂 /root/.cache/kagglehub/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail/versions/2/cnn_dailymail
   └── train.csv
   └── test.csv
   └── validation.csv

✅ Dataset loaded successfully!
Train shape: (287113, 3)
Validation shape: (13368, 3)
Test shape: (11490, 3)

Example article (truncated):
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attend

Reference summary:
Bisho

In [4]:
# Choose model checkpoint

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

model_checkpoint = 'facebook/bart-large-cnn'
# Alternatives: 't5-small', 'google/pegasus-cnn_dailymail'


print('Model checkpoint:', model_checkpoint)


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(DEVICE)


prefix = ''
if 't5' in model_checkpoint:
    prefix = 'summarize: '

Model checkpoint: facebook/bart-large-cnn


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [5]:
# Define preprocessing function


max_input_length = 1024
max_target_length = 128
if 't5' in model_checkpoint:
    max_input_length = 512
    max_target_length = 150


print('max_input_length =', max_input_length, 'max_target_length =', max_target_length)




def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')


    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=max_target_length, truncation=True, padding='max_length')


    labels_input_ids = labels["input_ids"]
    labels_input_ids = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels_input_ids]
    model_inputs["labels"] = labels_input_ids
    return model_inputs

max_input_length = 1024 max_target_length = 128


In [6]:
# Apply preprocessing to datasets


train_tokenized = train_ds.map(preprocess_function, batched=True, remove_columns=train_ds.column_names)
val_tokenized = val_ds.map(preprocess_function, batched=True, remove_columns=val_ds.column_names)


print(train_tokenized)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})


In [11]:
# Setup trainer


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


training_args = Seq2SeqTrainingArguments(
output_dir='./summarization_model',
# evaluation_strategy='epoch', # Removed as it's causing a TypeError
per_device_train_batch_size=2,
per_device_eval_batch_size=2,
predict_with_generate=True,
num_train_epochs=1,
logging_steps=200,
save_total_limit=2,
fp16=torch.cuda.is_available(),
)


trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=train_tokenized,
eval_dataset=val_tokenized,
tokenizer=tokenizer,
data_collator=data_collator,
)


print('Trainer ready')

Trainer ready


/tmp/ipython-input-1924745966.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [12]:
# Put model on correct device
model.to(DEVICE)
model.eval()

def generate_summary(article, **gen_kwargs):
    # Tokenize and move to device
    inputs = tokenizer([prefix + article], max_length=max_input_length, truncation=True, return_tensors="pt").to(DEVICE)

    # Generate summary
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example inference
example_article = test_df["article"].iloc[0]
reference = test_df["highlights"].iloc[0]

print("Original Article (truncated):", example_article[:400])
print("\nReference Summary:", reference)
print("\nGenerated Summary:", generate_summary(
    example_article,
    max_length=130,
    min_length=30,
    num_beams=4
))


Original Article (truncated): Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on 

Reference Summary: Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests conducted on planes with more leg room than airlines offer .

Generated Summary: U.S consumer advisory group set up by Department of Transportation said that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decr

In [13]:
# ROUGE evaluation


rouge = evaluate.load("rouge")


subset_articles = test_df["article"].iloc[:50].tolist()
subset_refs = test_df["highlights"].iloc[:50].tolist()


preds = [generate_summary(a, max_length=130, min_length=30, num_beams=4) for a in subset_articles]


results = rouge.compute(predictions=preds, references=subset_refs, use_stemmer=True)
results_percent = {k: round(v * 100, 2) for k, v in results.items()}
print("ROUGE:", results_percent)

KeyboardInterrupt: 

In [19]:
# Cell 10: Evaluate summaries with ROUGE
!pip install evaluate

import evaluate
import numpy as np

# Load ROUGE metric from new library
rouge = evaluate.load("rouge")

def evaluate_summaries(dataset, num_samples=100):
    articles = dataset["article"][:num_samples]
    references = dataset["highlights"][:num_samples]

    generated_summaries = []
    for article in articles:
        gen_sum = generate_summary(article, max_length=130, min_length=30, num_beams=4)
        generated_summaries.append(gen_sum)

    results = rouge.compute(predictions=generated_summaries, references=references, use_stemmer=True)

    # Print averaged scores
    for k, v in results.items():
        print(f"{k}: {v:.4f}") # Access the float value directly

    return results

# Evaluate on a subset for speed
rouge_results = evaluate_summaries(test_df, num_samples=5)

rouge1: 0.4378
rouge2: 0.2254
rougeL: 0.3196
rougeLsum: 0.3927


In [21]:
subset_articles = test_df["article"][:5]
subset_refs = test_df["highlights"][:5]

preds = []
for article in subset_articles:
    gen_sum = generate_summary(article, max_length=130, min_length=30, num_beams=4)
    preds.append(gen_sum)

In [22]:
# Save to CSV
out_df = pd.DataFrame({
    'article': [a[:500] + '...' for a in subset_articles],  # truncate for readability
    'reference': subset_refs,
    'prediction': preds,
})

out_df.to_csv('summaries_sample.csv', index=False)
print("Saved sample summaries to summaries_sample.csv")

Saved sample summaries to summaries_sample.csv
